In [1]:
import pickle
import os
import numpy as np
import pandas as pd
from pydub import AudioSegment
from tqdm.autonotebook import tqdm
import IPython.display as ipd
import librosa
import librosa.display
import glob, scipy, random
import noisereduce as nr

with open('model.sav', 'rb') as model:
    layers = pickle.load(model)
    weights = pickle.load(model)
    bias = pickle.load(model)
    longest = pickle.load(model)

C:\Users\alexp\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  


In [2]:
class NeuralNetwork:
    
    def __init__(self, layers):
        self.layers = layers
        self.weights = [None] * len(self.layers)
        self.bias = [None] * len(self.layers)
    
    def initialize(self, weights, bias):
        self.weights = weights
        self.bias = bias
    
    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))
 
    def softmax(self, z):
        exp = np.exp(z - np.max(z))
        return exp / exp.sum(axis=0, keepdims=True)

    def forward(self, X):
        a = X.T
        for i in range(1, len(self.layers)):
            z = self.weights[i] @ a + self.bias[i]
            if i == len(self.layers) - 1:
                a = self.softmax(z)
            else:
                a = self.sigmoid(z)
        return a
 
    def predict_write(self, X, names):
        pred = self.forward(X)
        answer = np.empty([X.shape[0], 6], dtype=object)
        answer[:,0] = answer[:,0].astype(str)
        answer[:,0] = names
        for i in range(5):
            answer[:,i + 1] = pred[i]
        
        df = pd.DataFrame(answer, columns=['name', '1', '2', '3', '4', '5'])

        df.to_csv(os.getcwd().replace("\\", "/") +'/8.csv', index=False)
        
            

In [3]:
nn = NeuralNetwork(layers)
nn.initialize(weights, bias)

In [4]:
path = os.getcwd().replace("\\", "/")
names = []
wavs = []

for audio, name in [(path + "/" + fname, fname) for fname in os.listdir(path)]:
    if name[-4:] != '.wav': continue
    names.append(name)
    data, sampling_rate = librosa.load(audio, sr = 16000)
    data = nr.reduce_noise(audio_clip=data, noise_clip=data)
    hpss = librosa.effects.hpss(data)
    data, ind = librosa.effects.trim(hpss[1], top_db=40)
    if longest >= len(data):
        data = list(data)
        data = [0.0] * int((longest - len(data))/2) + data + [0.0] * (longest - len(data) - int((longest - len(data))/2))
        data = np.array(data)
    else:
        speed_change = len(data) / float(longest)
        data = librosa.effects.time_stretch(data, speed_change)
    wavs.append(librosa.feature.mfcc(data, sr=16000).flatten())
try:
    X = np.zeros((len(wavs), len(wavs[0])))
    for i, dat in enumerate(wavs):
        X[i] = np.matrix(dat)
    X = X.astype(np.float64)
    nn.predict_write(X, names)
except:
    print("No test audiofiles in directory!!!")

C:\Users\alexp\Anaconda3\lib\site-packages\librosa\core\audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')
C:\Users\alexp\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: RuntimeWarning: overflow encountered in exp
  del sys.path[0]
